In [9]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Configurações para a simulação de dados meteorológicos
date_range = pd.date_range(start=datetime(2006, 1, 1), end=datetime(2006, 12, 31), freq='D')

# Simulando dados meteorológicos
weather_data = {
    "Date": date_range,
    "Tempestade": [random.choice([0, 1]) for _ in range(len(date_range))],
    "Visibilidade": [random.uniform(0.5, 10.0) for _ in range(len(date_range))],
    "Vento": [random.uniform(20, 120) for _ in range(len(date_range))]
}

# Criando o DataFrame
weather_df = pd.DataFrame(weather_data)

# Salvando o DataFrame simulado para uso posterior
weather_df.to_csv('dados_meteorologicos_2006.csv', index=False)

In [10]:
#Utilizar uma combinação do dataset de voos com o de dados meteorológicos

# Carregar o dataset de voos
flights_df = pd.read_csv('/home/roratto/study/facul/DS_Unesp/Datasets/2006-sample.csv')
climate_df = pd.read_csv('/home/roratto/study/facul/DS_Unesp/Work_Git/Pedro Roratto/WorkGit/dados_meteorologicos_2006.csv')

climate_df.head()
#flights_df.head()

,Date,Tempestade,Visibilidade,Vento
0,2006-01-01,0,6.484983,20.271588
1,2006-01-02,0,9.155873,102.745263
2,2006-01-03,0,7.679614,51.540888
3,2006-01-04,0,9.390305,45.181634
4,2006-01-05,0,9.012131,115.067566


In [11]:
#Definir a sazonalidade (Verão, Inverno, Outono, Primavera), e adicionar para cada registro do dataset de climatologia
#uma coluna indicando a qual estação do ano aquele registro pertence.
#Dica: Utilize a coluna Date para isso.
season = []
month = []
climate_df.Date = pd.to_datetime(climate_df.Date)
for i in climate_df.Date:
    month.append(i.month)
for i in month:
    if 3 <= i <= 5:
        season.append('Spring')
    elif 6 <= i <= 8:
        season.append('Summer')
    elif 9 <= i <= 11:
        season.append('Autumn')
    else:
        season.append('Winter')
climate_df['Season'] = season

# Criar uma coluna 'Season' no flights_df com base na coluna 'Month'
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

flights_df['Season'] = flights_df['Month'].apply(get_season)

# Calcular o atraso total (SumDelay) no flights_df
flights_df['SumDelay'] = flights_df.apply(
    lambda row: row['ArrDelay'] + row['DepDelay'] if (row['ArrDelay'] > 0 and pd.notna(row['ArrDelay']) and row['DepDelay'] > 0 and pd.notna(row['DepDelay'])) else 0,
    axis=1
)

# Agrupar os dados por 'Season' e calcular o atraso médio por estação
seasonal_delay = flights_df.groupby('Season')['SumDelay'].mean().sort_values(ascending=False)

# Exibir o ranking das estações do ano de acordo com o atraso médio em horas com até 2 casas decimais
print('Atraso médio por estação (em horas):')
print(seasonal_delay.round(2))

Atraso médio por estação (em horas):
Season
Summer    27.23
Winter    23.27
Autumn    23.24
Spring    20.90
Name: SumDelay, dtype: float64


In [12]:
def determine_flight_condition(row):
    if row['Visibilidade'] < 1 or row['Vento'] >= 103:
        return 'Não voe'
    elif row['Visibilidade'] < 2 or 61 <= row['Vento'] <= 102:
        return 'Condições péssimas'
    elif 2 <= row['Visibilidade'] < 5 or 41 <= row['Vento'] <= 61:
        return 'Condições normais'
    elif 5 <= row['Visibilidade'] < 8 or 20 <= row['Vento'] <= 40:
        return 'Condições boas'
    else:
        return 'Condições perfeitas'

climate_df['FlightCondition'] = climate_df.apply(determine_flight_condition, axis=1)

# Função para obter o impacto das condições meteorológicas em um dia específico
def get_weather_impact(year, month, day):
    # Filtrar o climate_df para a data específica
    date_str = f"{year}-{month:02d}-{day:02d}"
    weather_data = climate_df[climate_df['Date'] == date_str].iloc[0]
    weather_condition = weather_data['FlightCondition']
    tempestade = weather_data['Tempestade']
    visibilidade = weather_data['Visibilidade']
    vento = weather_data['Vento']
    
    # Filtrar o flights_df para a data específica
    flights_on_date = flights_df[(flights_df['Year'] == year) & (flights_df['Month'] == month) & (flights_df['DayofMonth'] == day)]
    
    # Calcular o atraso médio dos voos nesse dia
    avg_delay = flights_on_date['SumDelay'].mean()
    
    return weather_condition, avg_delay, tempestade, visibilidade, vento

# De para o usuário colocar os inputs do mês e ano
print("Digite o ano e o mês para verificar o impacto das condições meteorológicas (apenas 2006):")
day = int(input("Dia: "))
month = int(input("Mês: "))
year = 2006
#criar checagem para ver se a data existe
while True:
    try:
        datetime(year, month, day)
        break
    except ValueError:
        print("Data inválida. Digite uma data válida.")
        day = int(input("Dia: "))
        month = int(input("Mês: "))

weather_condition, avg_delay, tempestade, visibilidade, vento = get_weather_impact(year, month, day)
vento = round(vento, 2)
visibilidade = round(visibilidade, 2)
print(f"Condição meteorológica em {year}-{month:02d}-{day:02d}: {weather_condition}")
print(f"Parâmetros considerados - Tempestade: {tempestade}, Visibilidade: {visibilidade}, Vento: {vento} km/h")
print(f"Atraso médio dos voos: {abs(avg_delay):.2f} horas")

Digite o ano e o mês para verificar o impacto das condições meteorológicas (apenas 2006):
Condição meteorológica em 2006-10-10: Não voe
Parâmetros considerados - Tempestade: 1, Visibilidade: 4.15, Vento: 113.91 km/h
Atraso médio dos voos: 24.29 horas


In [17]:
# Analisar a correlação entre a condição de voo e o atraso médio diário
flights_df.rename(columns={'Year': 'year', 'Month': 'month', 'DayofMonth': 'day'}, inplace=True)

# Adicionar a condição de voo ao flights_df
flights_df['Date'] = pd.to_datetime(flights_df[['year', 'month', 'day']])
flights_df = flights_df.merge(climate_df[['Date', 'FlightCondition']], on='Date', how='left')
daily_analysis = flights_df.groupby(['Date', 'FlightCondition']).agg({'SumDelay': 'mean'}).reset_index()
correlation_analysis = daily_analysis.groupby('FlightCondition')['SumDelay'].mean().sort_values()

print('Correlação entre a condição de voo e o atraso médio diário (em horas):')
print(correlation_analysis)

Correlação entre a condição de voo e o atraso médio diário (em horas):
FlightCondition
Condições boas         21.062786
Condições péssimas     22.860750
Condições normais      23.450346
Não voe                24.976945
Condições perfeitas    53.457656
Name: SumDelay, dtype: float64
